This notebook initializes some quibits to 0 and then measures them, as a test for measurement errors.

In [3]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [5]:
# 2 cubit system
qc = qiskit.QuantumCircuit(2, 2)

# measure the qubits
qc.measure([0, 1], [0, 1])
qc = qiskit.transpile(qc, backend)

job = backend.run(qc, shots=10**4)
result = job.result()

/tmp/ipykernel_247040/310277187.py:8: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(qc, shots=10**4)


In [6]:
# display
counts = result.get_counts(qc)
print("Measurement output: ", counts)


Measurement output:  {'01': 21, '10': 24, '00': 9955}


In [9]:
# Again, but this time with 20 qubits
qc_20 = qiskit.QuantumCircuit(20, 20)
qc_20.measure([i for i in range(20)], [i for i in range(20)])
qc_20 = qiskit.transpile(qc_20, backend)

job_20 = backend.run(qc_20, shots=10**4)
result_20 = job_20.result()

/tmp/ipykernel_247040/1380385772.py:6: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job_20 = backend.run(qc_20, shots=10**4)


In [10]:
counts_20 = result_20.get_counts(qc_20)
print("Measurement output: ", counts_20)

Measurement output:  {'00000001100000000000': 19, '00110000100000000000': 1, '10001000000000000000': 2, '00000000100001000000': 3, '00010000000000100000': 1, '00000000000001010000': 3, '00000100000000000000': 13, '00100000001000000000': 7, '00001000000000010000': 3, '00001000100000010000': 1, '00000000010100000000': 1, '01000000100000000000': 3, '00001001000100000000': 1, '00110000001000000000': 1, '00001000001000000000': 24, '00000000101000000000': 10, '00000000001010000000': 2, '00000001001000000000': 8, '00100000001001000000': 1, '00000000001000000000': 187, '00000101000000000000': 2, '00000001000000001000': 2, '00000000000110000000': 1, '00010000000000001000': 2, '00010000100000010000': 1, '00100000000001000000': 5, '00110000000000000000': 6, '00000000001000000010': 1, '00000000100011000000': 1, '00000000100010000000': 3, '00000000000000001000': 13, '00000000100000100000': 2, '00000001000001000000': 2, '10100000000000000000': 1, '00100000000000000010': 1, '00101000100000000000': 2,

In [11]:
counts.get('0'*20, 0)

0

In [12]:
num_errors = {}
for measurement, count in counts_20.items():
    errors = sum([1 for i in range(20) if measurement[i] != '0'])
    num_errors[errors] = num_errors.get(errors, 0) + count
print(num_errors)

{2: 400, 3: 40, 1: 2409, 4: 1, 0: 7150}
